http://redux.js.org/  - "predictable state container for JavaScript apps
    -not 100% for react, but most commonly used for react. can use to handle state with any JS application

http://slides.com/devleague/redux

### redux is an implementation of flux (style of thinking how data flows)

normal react: parent <----> child //back and forth bidirectional dataflow (reverse flow)
    parent passes reference of State to child
    
redux is a state manager.
    - instaed of parent giving reference to state, child can trigger state method. 
    - parent doesn't have to pass state to child, listens to state changes from state manager. 
    - child is triggering Actions and redux holds entire state of the application and passes it to components

### three principles:
    - single source of truth
    - state is read-only
    - changes are made with pure functions
    
react is simple, reducers, middleware, store enhancers are fancy talk

**actions**: (think of it like an event) payloads of information that send data from your application to your store. 

**reducers**: determines how the state changes in response to an action

**store**: just an object. holds all of your state and brings **actions** and **reducers** together
    - holds application state
    - allows access to state via getSTate()
    - allows state to be updated via dispatch(action)
    - registeres listeners via subscribe(listener)
    - handles unregistering ...
    
redux is a **strict unidirectional data flow** thus data has the same lifecycle plan and logic can be easier to understand

time traveling with redux: since we don't mutate the values, you can go "back in time" to find what the state was at a specific point in time

### basic install for redux
- npm install --save redux 
- npm install --save react-redux

*in index.js(entry point) add*:  
import { createStore } from 'redux';  
import { Provider } from 'react-redux';

const store = createStore();

*next create two new directories: actions and reducers*  
    - mkdir root/src/actions ... root/src/reducers and add index.js files to each

# react notes
### container (smarty component)
- build containers when you need to do more than just rendering
- hold it's own state
- can take props (also via redux if implemented...2 ways to take props
- access to lifecycle methods: 
    -  componentWillMount
    -  conponentDidMount
    -  Render!!!
- we build as a class that extends React.Component


### component (dummy component)
- take props
- render JSX (not dom, html, etc... but JSX does get evaluated to markup)
- what is purpose? to render JSX


In [ ]:
//first setup the store
//entry point--- index.js

import { createStore } from 'redux';  
import { Provider } from 'react-redux';
import users from './reducers'; //this users comes in after you setup the reducer

const store = createStore(users);//add users back to the store once you do reducers

ReactDOM.render(
    <Provider store={store}>
        <App />
    </Provider>

)

In [ ]:
//then start making the actions
//src/action/users.js

export const ADD_USER = 'ADD_USER'

export const addUser = (user) => {
    return {
        type: ADD_USER,
        user: user
    }
}

In [ ]:
//then work on the reducers
//src/reducers//index.js

import { ADD_USER } from '../actions/users.js';

const initialState = { users: [] }

//action will always be an object with a type
const users = {state=initialState, action} {
    switch(action.type){
        case ADD_USER:
            return{
                users: [..state.users, action.user]//returns new state object back to Store
            }
        default:
            return state;
        }
    
}

export default users;

In [ ]:
//now in app.js
import { connect } from 'react-redux'//this allow you to connect to component store
import { addUser } from './actions/users.js'


//bottom 
const mapStateToProps = (state) =>{
    return{
        users: state.users
    }
}

//dispatch is a method by redux that allows components to invoke action methods/functions
//and send it through the pipeline to the reducers. 
//creating method in props object of class, takes argument and dispatches 

const mapDispatchToProps = (dispatch) =>{//allow componets to dispatch actions
    return{
        addUser: (user)=>{//addUser is name of the method in our this.props object
            dispatch(addUser(user))//this addUser is action function imported from above
        }
    }
}

const ConnnectedApp = connect(
    mapStateToProps,
    mapDispatchToProps
)(App)//connects App component to redux store
export default ConnectedApp;

In [ ]:
//sanity check ... in render(), console.log(this.props)

In [ ]:
//now create a UserList component
const UserList = ( {users} ) => {
  console.log('UserList component', users);
  return (
      <div>
        {
          users
        }
      </div>
  )
}

export default UserList;

In [ ]:
//then go back to App.js...
import UserList from './components/UserList.js'

class App extends Component {
  render() {
    console.log(this.props);
    return (
      <div>
        <h1>Hello World</h1>
        <UserList
          users={this.props.users}//gets the state from the Store
        />
      </div>
    );
  }
}

In [ ]:
//now back in UserLIst.js....
//this will work but we're going to break out the users as components after this...
const UserList = ( {users} ) => {
  console.log('UserList component', users);
  return (
    <div>
      {
        users.map(user =>{
          return (
            <li>
              { user }
            </li>
          )
        })
      }
    </div>
  )
}

export default UserList;

In [ ]:
//so lets break this out some more.... 

/////////UserItem.js
const UserItem = ( {name} ) =>{
  return (
    <li>
      { name }
    </li>
  )
}

/////////Updated UserList.js
const UserList = ( {users} ) => {
  console.log('UserList component', users);
  return (
    <div>
      {
        users.map(user =>{
          return (
            <UserItem name={user} />
          )
        })
      }
    </div>
  )
}

In [ ]:
//next we're going to add the "Submit" button 
//going to build it right in the App.js render as proof of concept, then later make a component
//#1- console log when the submit button is hit
//#2- keep track of user input, this will be stored in the local state NOT redux store/state
//#3- use constructor to be able to set LOCAL initial state
//#4- set LOCAL initial state and make a "userInput" state value
//#5- now you can set the state 
//#6- now when you hit submit, you can console.log the state

class App extends Component {
    constructor(props) {//#3 build constructor with initial state
        super();
    
    this.state={ //apps local state, not redux Store state
        userInput: ''//#4- adding a new state value called userInput
    }
  }
    
    
  submitUser(){
    console.log('hi');//#1 console logs when the submit button is clicked
    console.log(this.state.userInput);//#6 now you can console.log hte state
    this.props.addUser(this.state.userInput)//addProps here b/c of mapDispatchToProps adding it to dispatch
  }

  handleNewUserInput(e){
    console.log(e.target.value);//#2keeps track of data in submit box
    this.setState({
        userInput: e.target.value//#5 after initial state made in 4, you can set state 
    })
  }

  render() {
    console.log(this.props);
    return (
      <div>
        <h1>Hello World</h1>
        <UserList
          users={this.props.users}//gets the state from the Store
        />
        <input
          type="text"
          placeholder="new name"
          onChange={this.handleNewUserInput.bind(this)}
        />
        <button onClick={this.submitUser.bind(this)}>Submit</button>

      </div>
    );
  }
}

## reducers - 

#### index.js is main reducer, others are named to what they do

- move index.js to users.js, amake a new index.js
- each sub-reducer is going to be talking about the properties of the CR's object. the CR generates the object, the sub-reducer is what makes the values for that object. the key of the object comes from the export 
- when you use combinereducers, you don't need to return an entire object. CR returns the key to that value and you **can make an object of objects** b/c CR is making objects to put into the store. don't let this trip you up, look at the level of this.props for nesting
- whatever you export "export default varName", the varName will be the key for the exported array etc. in the props. 
- to update the state, you must return the same type of values back to the state. With CR, you are probably returning an array so you'll need to modify the return value to return an array as well (or object, or number, or whatever)

## async actions
- let redux ACTIONS handle data, get data, etc and let components render data. helps segregate as much as possible
- applyMiddleware is added to CreateStore so that any reducers are wrapped by the middleware and go through the dispatch
- actions --> middleware/redux-thunk --> reducers. when thunk is implemented, all actions will/must go through the middleware before getting to the reducers. if redux-think recognizes a function, it will **wait** until the function returns to dispatch and then pass the data on to the reducers. 

## react router
### front end routing != server side routing via express etc. 
- https://reacttraining.com/react-router/web/guides/philosophy
- https://github.com/ReactTraining/react-router
- react router allows us to render views (components) 
- big advantage is that by using < Link to> tag we don't have network requests and full page reloads. the page is only switching the views

In [ ]:
//setup express with /api/cards 
//  --- return JSON with all the available cards
//in KBcards --- hte KBcards will do a get request, addCardToDB will do a post requuest to /api/cards/new
//data store/in memory variable on the express server side...
//ideally no data on front end, all the data is on the server side and the front end does request to get it